In [1]:
from datetime import datetime
import modin.pandas as pd
from shapely import wkt
from shapely.geometry import Point

In [2]:
tweets = pd.read_json("tweets.jsonl", lines=True)
tweets = tweets[["id", "created_at", "coordinates"]]
tweets.set_index("id", inplace=True)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2024-01-21 23:01:03,861	INFO worker.py:1621 -- Started a local Ray instance.


In [3]:
forms = set(pd.read_parquet("wforms-ann.parquet", columns=[], filters=[
    [("status", ">=", 0)] # NOTE: this implies ( fst_batch | snd_batch ) & !tt_att
]).index)

tweets_toks = pd.read_parquet("tweets-tok.parquet", columns=["tokens"])
tweets_toks["tokens"] = tweets_toks["tokens"].apply(lambda ts: [t for t in ts if t in forms])
tweets_toks = tweets_toks[tweets_toks["tokens"].map(bool)]

tweets = tweets.join(tweets_toks, how="right")

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.
Reason: Parquet options that are not currently supported


In [4]:
tweets_geo = pd.read_parquet("tweets-geo.parquet", columns=["centroid"], filters=[[("id", "in", tweets.index)]])

tweets = tweets.join(tweets_geo, how="left")

Reason: Parquet options that are not currently supported


In [5]:
tweets["coordinates"] = tweets["coordinates"].apply(lambda o: None if o is None else Point(o["coordinates"]))
tweets["coordinates"] = tweets["coordinates"].fillna(tweets["centroid"].map(wkt.loads))

assert 477 == tweets["coordinates"].isna().sum() # sigh
tweets = tweets[~tweets["coordinates"].isna()]
tweets["latitude"] = tweets["coordinates"].apply(lambda p: p.y)
tweets["longitude"] = tweets["coordinates"].apply(lambda p: p.x)

In [6]:
tweets["created_at"] = tweets["created_at"].map(lambda dt: dt.isoformat())
tweets.reset_index(inplace=True)
tweets = tweets.explode("tokens")
tweets.rename(columns={"created_at": "timestamp", "tokens": "word"}, inplace=True)
tweets = tweets[["timestamp", "latitude", "longitude", "word"]]

In [7]:
tweets.to_csv("frustum-demo.csv", index=False)